<a href="https://colab.research.google.com/github/dwootton/Utah-Air-Explorers/blob/master/UtahAirExplorersClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import datetime
import requests
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive


In [9]:
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
sensor_info_df = pd.read_csv('drive/My Drive/UtahAirExplorers/sensorInformation.csv')

In [15]:
sensor_info_df

,Unnamed: 0,ID,Latitude,Longitude,Sensor Model,Sensor Source,Elevation
0,0,10050,40.749149,-111.912459,PMS5003+PMS5003+BME280,Purple Air,1288.745117
1,1,10051,40.749149,-111.912959,PMS5003,Purple Air,1288.989990
2,2,1010,40.847496,-111.901089,PMS5003+PMS5003+BME280,Purple Air,1349.093872
3,3,1011,40.847496,-111.901589,PMS5003,Purple Air,1346.343994
4,4,10132,40.449095,-111.760208,PMS5003+PMS5003+BME280,Purple Air,1553.029175
5,5,10133,40.449095,-111.760708,PMS5003,Purple Air,1551.906982
6,6,1018,41.026017,-111.901674,PMS5003+PMS5003+BME280,Purple Air,1414.564209
7,7,1019,41.026017,-111.902174,PMS5003,Purple Air,1411.444458
8,8,1024,41.327040,-111.814401,PMS5003+PMS5003+BME280,Purple Air,1669.746460
9,9,1025,41.327040,-111.814901,PMS5003,Purple Air,1669.021484


In [16]:
sensor_info_df.describe()


,Unnamed: 0,Latitude,Longitude,Elevation
count,787.000000,787.000000,787.000000,787.000000
mean,393.000000,40.639899,-111.845809,1492.469644
std,227.331623,0.528685,0.412879,236.617116
min,0.000000,37.111172,-114.042201,900.597656
25%,196.500000,40.478486,-111.931949,1346.753357
50%,393.000000,40.684050,-111.861434,1438.064453
75%,589.500000,40.777160,-111.768597,1540.261719
max,786.000000,41.766739,-109.490258,2712.012939


It looks like some of our sensors aren't where we want them to be. Let's try to narrow down our sensors to only the Salt Lake Valley. This should get rid of outliers. 

In [0]:
sensor_info_cleaned = sensor_info_df[sensor_info_df['Latitude']> 40.5]
sensor_info_cleaned = sensor_info_cleaned[sensor_info_cleaned['Latitude']< 40.9]
sensor_info_cleaned = sensor_info_cleaned[sensor_info_cleaned['Longitude']> -112.01]
sensor_info_cleaned = sensor_info_cleaned[sensor_info_cleaned['Longitude']< -111.75]

In [22]:
salt_lake_sensors = sensor_info_cleaned
salt_lake_sensors.describe()

,Unnamed: 0,Latitude,Longitude,Elevation
count,308.000000,308.000000,308.000000,308.000000
mean,435.928571,40.697870,-111.866966,1404.891104
std,251.817151,0.084157,0.054492,99.868619
min,0.000000,40.507019,-112.000040,1286.633301
25%,220.750000,40.645648,-111.900736,1312.909210
50%,411.500000,40.712139,-111.867127,1379.080078
75%,704.250000,40.755075,-111.819422,1484.098999
max,786.000000,40.891716,-111.765375,1675.956421
